In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import ImageDataset, ConvResNet, init_dataloader, channel_binary_mask, \
    checkerboard_binary_mask, AffineCouplingLayer, RealNVP, Scale
import torch
from torch.distributions.normal import Normal
from math import log
device = torch.device("cuda")

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [3]:
train_dataset = ImageDataset(root_dir="/home/ext_amin_unity3d_com/real_nvp_data/train")
train_dataloader = init_dataloader(train_dataset)

In [6]:
# realnvp = RealNVP().to(device)
# optimizer = torch.optim.Adam(lr=0.01, params=realnvp.parameters())

# epochs = 3
# for e in range(epochs):
#     for i, data in enumerate(train_dataloader):
#         image_batch = data.to(device)
#         optimizer.zero_grad()
#         loss, z = realnvp(image_batch)
#         loss.backward()
#         optimizer.step()
#         if i % 99 == 0:
#             print(f"epcoh {e}, batch {i}, loss {loss}")
        
#     print(f"epoch {e}, loss: {loss}")
    

In [4]:
tmp_input = torch.ones(1, 3, 32, 32).to("cuda:0")

In [5]:
aff_1 = AffineCouplingLayer(
    mask_type="checkerboard_binary_mask",
    mask_orientation=1,
    input_shape=[3, 32, 32],
    num_filters=64).to("cuda:0")

aff_2 = AffineCouplingLayer(
    mask_type="checkerboard_binary_mask",
    mask_orientation=0,
    input_shape=[3, 32, 32],
    num_filters=64).to("cuda:0")

In [6]:
forwarded_1, _ = aff_1(tmp_input)
forwarded_2, _ = aff_2(forwarded_1)

backward = aff_1.reverse(aff_2.reverse(forwarded_2))

print(torch.allclose(tmp_input, backward))
print(torch.norm(tmp_input - backward))

masked_x tensor([[[1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         ...,
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.]],

        [[1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         ...,
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.]],

        [[1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         ...,
         [0., 1., 0.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 1., 0., 1.]]], device='cuda:0')
masked_x tensor([[[ 0.0000,  2.8496,  0.0000,  ...,  0.4978,  0.0000,  1.5328],
         [ 2.4485,  0.0000, -0.3182,  ...,  0.0000, -0.3959,  0.0000],
         [ 0.0000, -0.4481,  0.0000, 

In [8]:
scale = Scale().to("cuda")
forwarded, _ = scale(tmp_input)
backward = scale.reverse(forwarded)
torch.allclose(tmp_input, backward)
# torch.norm(tmp_input - backward)

masked_x tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], device='cuda:0')
masked_x tensor([[[[-0.1222,  0.0000,  0.3991,  ...,  0.0000,  0.7031,  0.0000],
          [ 0.0000,  0.8039, -0.0000,  ..., -2.3993, -0.0000,  1.5325],
          [-0

False

In [21]:
torch.norm(tmp_input - backward)

tensor(0.7936, device='cuda:0', grad_fn=<NormBackward1>)